In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

from modules.mail import send_checks_mail

import sys
from config.variables import user

In [2]:
training_code = 'ARP'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CHC', 'AUX'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
19,00000058,SARAIVA,CHC,IRX,GIG
20,00000070,ADRI MICHELLI,CHC,IR,GRU
21,00000071,ALINE,CHC,NaN,CGH
22,00000072,ANA PAGLIARO,CHC,IRX,CGH
24,00000080,FABI FREITAS,CHC,NaN,CGH
...,...,...,...,...,...
5454,00046322,JULIA CASTILHO,AUX,NaN,CGH
5462,00046334,MARIANE PAIVA,AUX,NaN,CGH
5463,00046335,THAYNA NASCIMENTO,AUX,NaN,CGH
5464,00046341,GISELLE LUGAO,AUX,NaN,CGH


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}_tripulacao_comercial.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
2433,00000000,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Em andamento,"0,00",NaT
766,00000058,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"8,00",2020-07-08 17:03:01
2536,00000070,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Reprovado,"7,00",2020-06-02 18:29:59
2393,00000072,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2020-11-20 12:02:23
1702,00000080,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2020-12-11 14:47:44
...,...,...,...,...,...
363,00045150,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,00",2020-10-09 09:52:12
43,00045167,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2020-10-09 14:26:57
1544,00045500,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2021-01-06 11:30:34
1993,00046258,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Em andamento,"0,00",NaT


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
169949,00035164,INSS
169950,00035169,INSS
169959,00032868,INSS
169963,00032709,INSS
169965,00032174,INSS
...,...,...
171213,00002329,INSS
171216,00002149,INSS
171227,00000866,INSS
171231,00004863,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
169913,00046310,LIC-SREM
169914,00046322,LIC-SREM
169915,00046327,LIC-SREM
169916,00046330,LIC-SREM
169917,00046341,LIC-SREM
...,...,...
171224,00001154,LIC-SREM
171225,00001160,LIC-SREM
171228,00005227,LIC-SREM
171232,00004746,LIC-SREM


In [9]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

,cif,code,fr,dep
152211,00000019,C-ARP-ON,CGH,2020-06-29 16:00:00
38675,00000020,C-ARP-ON,CGH,2020-06-08 16:00:00
60540,00000021,C-ARP-ON,CGH,2020-12-10 16:30:00
33015,00000024,C-ENS-ARP,CGH,2020-05-07 16:30:00
105381,00000026,C-ARP-ON,CGH,2020-12-17 16:15:00
...,...,...,...,...
76481,00046079,C-ENS-ARP,CGH,2020-03-11 11:15:00
76325,00046080,C-ENS-ARP,CGH,2020-03-11 11:15:00
76306,00046081,C-ENS-ARP,CGH,2020-03-11 11:15:00
76397,00046082,C-ENS-ARP,CGH,2020-03-11 11:15:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
79,00005309,ARP,2021-05-26
84,00024283,ARP,2021-06-30
89,00010655,ARP,2019-12-31
102,00003256,ARP,2021-06-18
118,00003659,ARP,2021-08-22
...,...,...,...
58510,00045803,ARP,2021-02-17
58511,00045793,ARP,2021-02-17
58512,00045777,ARP,2021-02-17
58513,00045774,ARP,2021-02-17


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr
0,00000058,SARAIVA,CHC,IRX,GIG,ARP,2021-07-08,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"8,00",2020-07-08 17:03:01,C-ARP-ON,2020-07-27 16:15:00,NaN,NaN
1,00000070,ADRI MICHELLI,CHC,IR,GRU,ARP,2021-06-09,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Reprovado,"7,00",2020-06-02 18:29:59,C-ENS-ARP,2020-06-11 16:15:00,NaN,NaN
2,00000071,ALINE,CHC,NaN,CGH,ARP,2020-09-24,NaN,NaN,NaN,NaT,NaN,NaN,INSS,NaN
3,00000072,ANA PAGLIARO,CHC,IRX,CGH,ARP,2021-11-20,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2020-11-20 12:02:23,C-ARP-ON,2020-11-20 16:31:00,NaN,NaN
4,00000080,FABI FREITAS,CHC,NaN,CGH,ARP,2021-12-11,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"9,50",2020-12-11 14:47:44,C-ARP-ON,2020-12-11 16:31:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,00046322,JULIA CASTILHO,AUX,NaN,CGH,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,LIC-SREM
3443,00046334,MARIANE PAIVA,AUX,NaN,CGH,ARP,2021-09-09,NaN,NaN,NaN,NaT,NaN,NaN,NaN,LIC-SREM
3444,00046335,THAYNA NASCIMENTO,AUX,NaN,CGH,ARP,2021-06-17,NaN,NaN,NaN,NaT,NaN,NaN,NaN,LIC-SREM
3445,00046341,GISELLE LUGAO,AUX,NaN,CGH,ARP,2021-05-25,NaN,NaN,NaN,NaT,NaN,NaN,NaN,LIC-SREM


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

#validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)
validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
118,00000821,MARCELO RECHE,CHC,IRX,CGH,ARP,2021-01-26,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-27,NaN,NaN,-5451,False
617,00004567,NANDA FERNANDES,AUX,NaN,CGH,ARP,2021-01-20,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-21,NaN,LIC-SREM,-5451,False
817,00006284,LUCIANA LIMA,AUX,NaN,GRU,ARP,2021-01-13,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-14,NaN,LIC-SREM,-5451,False
1101,00009041,LETICIA VIDAL,AUX,NaN,CGH,ARP,2021-01-19,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-20,NaN,LIC-SREM,-5451,False
1330,00011808,MARCELO FERNANDES,CHC,NaN,FOR,ARP,2021-01-07,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-08,NaN,LIC-SREM,-5451,False
1920,00022694,JULIO BONATO,CHC,NaN,BSB,ARP,2021-01-21,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"10,00",2021-01-22,C-ARP-ON,2021-01-22,NaN,NaN,5,True
2018,00023221,BIA COIMBRA,AUX,NaN,GRU,ARP,2021-01-06,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-07,INSS,NaN,-5451,False
2546,00035066,FRANCIELLE LIMA,AUX,NaN,BSB,ARP,2021-01-21,NaN,NaN,NaN,2035-12-31,C-ENS-ARP,2020-01-22,INSS,NaN,-5451,False


In [13]:
validation.to_csv(f'./data/output/{training_code}_tripulacao_comercial.csv', index=False, sep=';', encoding='utf-8-sig')

In [14]:
# Arquivo de import
# fill table
df_atualizar = validation[validation['atualizar']]

df_atualizar

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
1920,00022694,JULIO BONATO,CHC,NaN,BSB,ARP,2021-01-21,DGR - PERIODICO - ARTIGOS PERIGOSOS - CHAVE 11...,Concluído,"10,00",2021-01-22,C-ARP-ON,2021-01-22,NaN,NaN,5,True


In [17]:
df_import = df_atualizar[['cif', 'checkTypeCode', 'data_conclusao']]

# fill table

data = df_import.copy()

data['update'] = 'U'
data['id'] = df_import['cif'].astype(str).str.zfill(8)
data['type'] = 'ARP'
data['issue'] = df_import['data_conclusao'].dt.strftime('%Y%m%d') + '0000'
data['expiry'] = (df_import['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['extended'] = (df_import['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['failed'] = 'N'
data['do_not_plan_before'] = (df_import['data_conclusao'] + td(days=365)).dt.strftime('%Y%m%d') + '0000'
data['zero'] = '0'
data['remark'] = ''
data['check_comment'] = ''
data['confirmed'] = 'Y'
data['paper_recieved'] = 'Y'

# remove fields from original file
cols = [
    'update',
    'cif',
    'type',
    'issue',
    'expiry',
    'extended',
    'failed',
    'do_not_plan_before',
    'zero',
    'remark',
    'check_comment',
    'confirmed',
    'paper_recieved'  
]

data = data[cols].copy()

data.to_csv('./data/output/crm_import_arp.dat', sep='|', header=None, index=False, line_terminator='\r\n')

data

,update,cif,type,issue,expiry,extended,failed,do_not_plan_before,zero,remark,check_comment,confirmed,paper_recieved
1920,U,00022694,ARP,202101220000,202201220000,202201220000,N,202201220000,0,,,Y,Y


In [18]:
out_email = validation.copy()

out_email = out_email[~out_email['inss'].isin(['INSS'])]
out_email = out_email[~out_email['lnr'].isin(['LIC-SREM'])]

out_email = out_email[['cif',
                       'guerra',
                       'funcao',
                       'base',
                       'checkTypeCode',
                       'expiryDt', 
                       'status', 
                       'nota', 
                       'data_conclusao',
                       'code',
                       'dep',
                       'tempo_de_conclusao (dias)',
                       'atualizar']]

out_email['expiryDt'] = out_email['expiryDt'].dt.strftime('%d/%m/%Y')
out_email['data_conclusao'] = out_email['data_conclusao'].dt.strftime('%d/%m/%Y')
out_email['dep'] = out_email['dep'].dt.strftime('%d/%m/%Y')
out_email['status'] = out_email['status'].fillna('')
out_email['nota'] = out_email['nota'].fillna('')


In [19]:
send_checks_mail(out_email, user['mail'], user['password'], user['emails'])